# 📊 Sistema de Informação de Mortalidade - Banco de Dados Relacional

##  Introdução

Este projeto tem como objetivo a criação de um **banco de dados relacional** para armazenar e analisar informações sobre óbitos, utilizando dados extraídos do Sistema de Informação de Mortalidade (SIM). A implementação inclui **ETL**, **views**, **procedures**, **triggers** e diversas consultas SQL para explorar os dados.

## Modelo de Dados Relacional

O banco de dados foi estruturado seguindo os princípios da **modelagem relacional**, garantindo **integridade referencial** e **normalização**. O esquema contém as seguintes tabelas:

- **`obitos`** - Registros dos óbitos com detalhes como local, hora, causa da morte, entre outros.
- **`localizacoes`** - Municípios e estados onde os óbitos ocorreram.
- **`causas`** - Código CID e descrição das causas de morte.


Diagrama do Modelo Relacional:

![Diagrama ER](link_para_diagrama.png)

##  Script SQL - Criação do Banco de Dados

O banco de dados foi criado utilizando PostgreSQL. O script SQL para criação das tabelas está disponível no arquivo [`mortalidade.sql`](mortalidade.sql).




## Processo ETL - Importação dos Dados
O processo ETL (Extract, Transform, Load) foi implementado em Python, utilizando as bibliotecas pandas e SQLAlchemy para:

Extração: Leitura dos arquivos CSV contendo os registros de óbitos.
Transformação: Limpeza e padronização dos dados (conversão de datas, remoção de inconsistências, tratamento de valores nulos).
Carga: Inserção dos dados processados no banco PostgreSQL.
Exemplo de código para transformação da coluna idade:

##  Utilização de View


##  Utilização de Procedure


##  Utilização de Trigger


# Consultas SQL

Abaixo estão cinco consultas SQL explorando os dados do banco:

## Quantidade de óbitos por município (TOP 10)

```sql
SELECT l.municipio, COUNT(o.id_obito) AS total_obitos
FROM obitos o
JOIN localizacoes l ON o.id_local = l.id_local
GROUP BY l.municipio
ORDER BY total_obitos DESC
LIMIT 10;
```



# Consultas em Álgebra Relacional

##  Quantidade de Óbitos por Município (TOP 10)

$$
\pi_{\text{municipio}, \text{total\_obitos}} \left( \sigma_{\text{total\_obitos} \geq 10} \left( \gamma_{\text{municipio}}^{\text{COUNT}(id\_obito) \rightarrow \text{total\_obitos}} \left( \text{Obitos} \bowtie_{\text{Obitos.id\_local} = \text{Localizacoes.id\_local}} \text{Localizacoes} \right) \right) \right)
$$

### **Explicação**:
- **\(⨝\)** → Junção entre **Óbitos** e **Localizações** baseada no `id_local`.
- **\(γ\)** → Agrupa por município e conta o número de óbitos.
- **\(σ\)** → Filtra municípios com pelo menos **10 óbitos**.
- **\(π\)** → Retorna apenas as colunas **município** e **total_obitos**.

---

## CID com Maior Número de Mortes

$$
\pi_{\text{codigo\_cid}, \text{descricao}, \text{total\_obitos}} \left( \sigma_{\text{total\_obitos} = \max(\text{total\_obitos})} \left( \gamma_{\text{codigo\_cid}, \text{descricao}}^{\text{COUNT}(id\_obito) \rightarrow \text{total\_obitos}} \left( \text{Obitos} \bowtie_{\text{Obitos.id\_causa\_principal} = \text{Causas.id\_causa}} \text{Causas} \right) \right) \right)
$$

###  **Explicação**:
- **\(⨝\)** → Junção entre **Óbitos** e **Causas** baseada no `id_causa`.
- **\(γ\)** → Agrupa por **CID** e conta o número de óbitos.
- **\(σ\)** → Filtra apenas o CID com **maior número de mortes**.
- **\(π\)** → Retorna **código CID**, **descrição** e **total de óbitos**.

---

## Média de Idade dos Falecidos por Região

$$
\pi_{\text{regiao}, \text{media\_idade}} \left( \gamma_{\text{regiao}}^{\text{AVG}(idade) \rightarrow \text{media\_idade}} \left( \text{Obitos} \bowtie_{\text{Obitos.id\_local} = \text{Localizacoes.id\_local}} \text{Localizacoes} \right) \right)
$$

### **Explicação**:
- **\(⨝\)** → Junção entre **Óbitos** e **Localizações** baseada no `id_local`.
- **\(γ\)** → Agrupa por **região** e calcula a média da idade.
- **\(π\)** → Retorna apenas as colunas **região** e **média de idade**.

---

 Notação formal de **Álgebra Relacional**:
- **Seleção** \(σ\)
- **Projeção** \(π\)
- **Junção** \(⨝\)
- **Agregação** \(γ\)


